# Constructors

%- TODO: Empty constructors. Implicit, explicit.


## Motivation

Constructors solve two main problems.

1. They allow us to initialize the [fields](fields) (and [properties](properties) but we haven't talked about those yet) of an object upon creation.
2. They allow us to validate whether it, in the current state of the world and given the data that we have been passed, is possible to create an instance of the object or not. This problem can generally be thought of as "validation" or "sanitization".

The second point is very important since it allows us to assume that any instance of a type actually is a valid instance of that type.
Remember the maxim of making impossible states impossible in the chapter on [types over tests](types-over-tests)?
We can use constructors to make sure that our application never contains an instance (meaning value) that's nonsensical given the definition of that type.

Take the notion of vowels for example.
According to the [bijection rule](types-over-tests) or the notion of types over tests, it is not sensible to use a `char` to model vowels.
The two data types support different values and different operations.


## Definition

A constructor, in C#, is any method whose name is the same as the enclosing type (in this case: class) and that doesn't specify a return type.
There is no need to specify a return type because a constructor always returns instances of the type in which it is defined.

```
class [identifier]
{
  public [identifier] (<[parameters]>)
  {
  }
}
```

In the chapter on [classes](classes) we discussed the keyword `new` which is used to create instances of a class.
When we use the keyword `new` followed by a class name, a pair of parentheses, and possibly some arguments, then a constructor (that matches that list of arguments is called) is called and we get back an object of that type.
I say "a constructor" rather than "the constructor" since constructors support overloading and so a class might have any number of constructors.
More on that in a moment though.

%Let's write a constructor for the `Balloon` class that we discussed in the chapter on [classes](classes).

### Default constructor

If we don't define a constructor for a class in C# then it's as if that class contains a constructor that takes no parameters and has no method body.
As soon as we define a constructor though, this empty and parameterless constructor is no longer implicitly created.

Remember how we, in the chapter on [fields](fields), mentioned that instance fields in C# are automatically intialized to their default values unless we initialize them ourselves in the class or upon construction using an [object initializer](object-initializers)?

Let's say that we've got a class that models a point on a two-dimensional grid.

In [1]:
class Point
{
  public int X = 0; // Default value.
  public int Y = 0; // Default value.
}

The below script needs to be able to find the current output cell; this is an easy method to get it.

Whenever we create an instance of the class `Point`, this empty and parameterless "default" constructor is run.

In [2]:
Point p1 = new Point(); // This causes the constructor to be called.

Since the empty constructor isn't changing the value of any of the instance fields, the instantiated point will contain whatever values we set as defaults.
As can be observed below.

In [3]:
Console.WriteLine(p1.X);

0


### Initializing fields

What's the problem?
Well, sometimes it's possible to identify a sensible "default" value that we can initialize our instance fields to, and sometimes it's not.
What if we don't want to specify a default value for a field in our class?
Worse, what if there really isn't any default value that remotely makes sense in our domain?
Remember the maxim of [making impossible states impossible](types-over-tests).
If it doesn't make sense in the domain, then we shouldn't allow it in our types.

By declaring a constructor with some parameters we make it impossible to instantiate the class without also passing arguments for those parameters.
Remember what we said about the default constructor.
An empty, parameterless constructor isn't implicitly added to your class if you define a constructor.
However, to construct an instance of a class, we must run a constructor.
So, if there is only one constructor in a class, and that constructor takes some parameters, then it is impossible to create an instance of that class without also providing arguments for those parameters.

By using constructors we can force anyone who wants to create an instance of the class to have to specify the two coordinates that the point should be at.
It would look like this.

In [4]:
class Point
{
  public int X; // No default needed.
  public int Y; // No default needed.

  // This is the constructor.
  public Point (int x, int y)
  {
    this.X = x; // Assigns input argument to field.
    this.Y = y; // Assigns input argument to field.
  }
}

```{note}
Wondering what the keyword `this` is?
Don't worry, we'll get there in a second.
```

In the code above, we've declared a constructor with two parameters of type `int`.
The two parameters are named `x` and `y`.
In the body of the constructor, meaning the implementation of the constructor, we take these two arguments and assign them to our instance fields called `X` and `Y`.

When do we pass these arguments?
We pass them when we instantiate the class.

In [5]:
Point p2 = new Point(10, 20);
Point p3 = new Point(4, 5);

Notice how it is no longer possible to instantiate a `Point` without also passing arguments to the constructor.
If we try to do that, then we get a compiler error.

In [6]:
Point p4 = new Point(); // Will not compile.


(1,16): error CS7036: There is no argument given that corresponds to the required formal parameter 'x' of 'Point.Point(int, int)'



Error: compilation error

A constructor is a method, and to call the method we must supply arguments for all its parameters.

This means that we can guarantee that anytime we have an instance of type `Point` it was instantiated with some intial values that whoever instantiated the class chose.
However, we can *not* guarantee that any instance of type `Point` will at all times have coordinates that "make sense" in our domain since the [access modifiers](access-modifiers) of the instance fields are `public` and since the fields are [mutable](mutability).
To achieve true information hiding we must also learn about [instance methods](instance-methods) or [properties](properties).


### `this` keyword

So what about the `this` keyword that's being used in the example above.
What is it and what does it do?

When the code in the body of the constructor is being executed, then that code is being executed in the context of the object that was just created.
The keyword `this` refers to the object that is being constructed.

In that particular example we don't actually need the keyword `this`.
We could just as well, simply, have written `X = x` and `Y = y`.
This is because there is no ambiguity.
What do we mean when we say that there is no ambiguity?
We mean that the variables `X` and `x`, for example, are both only defined in one place that's in scope in the constructor.

If however we had named the constructor parameters `X` and `Y`, meaning in uppercase then there would be ambiguity.
Think about it.
What are we assigning to what if we would say `X = X`?
The C# compiler will assume that we are referring to the local variable which means that we will be assigning the value of a variable to itself.
Which of course is pointless and not what we meant to do.
Have a look at the compiler warnings that we get below:

In [7]:
class Point
{
  public int X = 0;
  public int Y = 0;

  public Point (int X, int Y)
  {
    X = X;
    Y = Y;
  }
}


(8,5): warning CS1717: Assignment made to same variable; did you mean to assign something else?

(9,5): warning CS1717: Assignment made to same variable; did you mean to assign something else?



%TODO: WHAT DOES THE KEYWORD `THIS` MEAN


```{warning}
Work in progress.
```


### Overloading

Constructors are [overloadable](overloading).
This is known as "constructor overloading" and means that we can define multiple constructors that differ in their parameter lists and have different implementations.
This is very useful since it, for example, allows us to define types that can be instantiated in very different ways.

Consider for example the idea of a weekday.
If we have a type that models weekdays called `Weekday` then we can trivially think of a number of different ways to instantiate it.
To instantiate an object that represents Tuesday we might for example want to support the following different syntaxes.

In [8]:
class Weekday
{
  public Weekday (string day) {}
  public Weekday (int day) {}
}

In [9]:
new Weekday (2);            // int
new Weekday ('T');          // char
new Weekday ("Tuesday");    // string

Notice how we're instantiating the same type, namely `Weekday`, in all three cases above.
However, notice also how the type of the argument that we pass to the constructor varies.
There are essentially three constructors.

In [10]:
class Weekday
{
  public Weekday (string day) { /* ... */ }
  public Weekday (int day) { /* ... */ }
  public Weekday (char day) { /* ... */ }
}

Whichever constructor is run depends on the data type we pass when we instantiate the class.


### Constructor chaining

```{warning}
Work in progress.
```


### Access modifiers


```{warning}
Work in progress.
```

%- TODO: Constructors can be private.
%- TODO: Example: Use static methods to allow us to only create "correct" vowel instances. Prefer compile-time errors over run-time errors.


### Expression-bodied members

As mentioned in an earlier chapter, constructors support the C# feature known as expression-bodied members.
This means that if we have a constructor whose implementation can be expressed as a single expression, then we can omit the curly braces (`{ }`) and instead use a fat-arrow (`=>`).
Like in the example below.

In [11]:
class Person
{
  public string Name;
  public Person (string name)
    => Name = name;
}

%### Order
%
%First field initializers, then chained constructor, then constructor?


## Examples

%### Vowels
%
%```{code-cell} csharp
%class Vowel
%{
%  char letter;
%
%  public Vowel (char vowel)
%  {
%    string legal = "AEIOU";
%    if (legal.IndexOf(Char.ToUpper(vowel)) == -1)
%      throw new ArgumentException("Must be a vowel (AEIOU).");
%    letter = vowel;
%  }
%
%  public char ToChar () => letter;
%
%  public static implicit operator char(Vowel v) => v.ToChar();
%}
%```
%Write a class that models the domain concept of English vowels.
%It should be possible to pass upper and lowercase vowels as characters when instantiating it, and it should be possible to extract these vowels again.
%See the usage examples below.
%
%```{code-cell} csharp
%char c1 = new Vowel('A').ToChar();
%Console.WriteLine(c1);
%```
%
%```{code-cell} csharp
%char c2 = new Vowel('u').ToChar();
%Console.WriteLine(c2);
%```
%
%Rember the maxim [types over tests](types-over-tests)?
%It suggests that illegal states shouldn't be representable.
%If you try to instantiate the class by passing a non-vowel character then the class should throw an `ArgumentException`.
%
%```{code-cell} csharp
%:tags: [raises-exception]
%new Vowel('z').ToChar();
%```
%
%
%We have learned that illegal states should not be representable.
%We have also learned that compile-time errors are preferable over run-time errors.
%Rewrite the class `Vowel` that you wrote in {numref}`ex:constructors:vowel` so that we get a compile-time error instead of a run-time error whenever we try to instantiate it with a non-vowel.
%
%Hint: Use [enums](enums).


### Robber's cipher vowel

In the chapter on [fields](fields) we gave an example where we assigned the consonant `B` to a field that was supposed to contain a vowel.
Let's add a constructor to that class so that we can ensure that every time we have an instance of the class `RobbersCipherVowel` we know that it contains an actual vowel.

In [12]:
class RobbersCipherVowel
{
  public char Vowel;
  public RobbersCipherVowel (char vowel)
  {
    Vowel = vowel switch {
      'A' or 'a' or 'E' or 'e' or 'I' or 'i' or 'O' or 'o' or 'U' or 'u'
        => vowel,
      _ => 'o'
    };
  }
}

Pretty neat right?
What happens if we now try to create an instance of `RobbersCipherVowel` by passing a consonant (or any other non-vowel character for that matter) to the constructor?
Well, the constructor will simply ignore the character that we sent it and instead just use the character `'o'` that we chose as the default.

In [13]:
RobbersCipherVowel certainlyVowel = new RobbersCipherVowel('K');
Console.WriteLine(certainlyVowel.Vowel);

o


%Arguably, it would make sense to use an [enum](enums) for input instead.
%We're not going to write that code since it's given to you as {numref}`ex:constructors:vowel-enums`.
%However, we're going to talk a lot more about the idea of using types that prevent your data types from ever entering illegal states in the chapter on [types over tests](types-over-tests).

But wait a minute!
What if someone just changes the vowel *after* the object has been created?

```{danger}
Be weary of mutating fields and properties.
```

In [14]:
certainlyVowel.Vowel = 'K';
Console.WriteLine(certainlyVowel.Vowel);

K


That's definitly not a vowel!
This is a problem caused by allowing mutation and not actually embracing encapsulation.
We're actually not encapsulating the vowel.
We're not actually hiding it from the outside world.
When we say that it is `public` we've exposed it.
And since it's a field, that means that anyone can change it.

To encapsulate the vowel we need to learn about [instance methods](instance-methods) or [properties](properties).


(exception-arguments)=
### Exceptions

Remember how we discussed that [expections](throwing-exceptions) are just regular classes and that we can instantiate and then throw them ourselves?
There are two things we must learn about exceptions when talking about constructors.

1. We can pass arguments to the constructor when instantiating an exception.
2. We can throw exceptions in our own constructors when users try to create instances of our type that doesn't make sense.

If an exception is just a class, then that means that when we are instantiating them we are running a constructor.
Which in turn means that the constructor might take arguments.

I've mentioned that we'll make heavy use of the type `ArgumentException`, and it just so happens that this class, as many other exception classes in .NET, defines a constructor that takes a single `string` as an argument.
This `string` is the message that we want to send with the exception when it is thrown.

We can use this feature in order to tell the user more about what went wrong when something went wrong.
Have a look at how we're using exceptions below to guarantee that a Balloon without a text cannot be created.

In [15]:
class Balloon
{
  public string Text = "";

  public Balloon (string text)
  {
    if (text == null || text == "")
      throw new ArgumentException($"Text must be non-null and non-empty.");
    this.Text = text;
  }
}

### Static factory methods

```{warning}
Work in progress.
```

%!!!!!!!!!!!!!!!!!!!!!
%!!!!!!!!!!!!!!!!!!!!!
%!!!!!!!!!!!!!!!!!!!!!
%!!!!!!!!!!!!!!!!!!!!!
%VOWEL EXAMPLE AND STATIC FACTORIES



## Exercises

```{exercise}
How can we make use of the keyword `this` in a constructor?
```

```{exercise}
What is constructor overloading and constructor chaining?
How do they relate to eachother?
```

```{exercise}
Write your own example that shows how constructor overloading works.
```

```{exercise}
Write your own example that shows how constructor chaining.
```

```{exercise-start}
```
The code below generates warnings and doesn't behave as intended.
How do we get rid of the warning and make it behave as the programmer most likely intended it to behave like?

In [16]:
class Cat
{
  string Name;

  public Cat (string Name)
    => Name = Name;
}


(6,8): warning CS1717: Assignment made to same variable; did you mean to assign something else?

(3,10): warning CS0169: The field 'Cat.Name' is never used



```{exercise-end}
```


%```{exercise}
%:label: ex:constructors:vowel-enums
%
%Use constructor overloading to add another constructor to the class `CaesarCipherVowel` from this chapter.
%The new constructor should take an [enum](enums) that only can be set to vowels.
%Make sure that the updated class still can be used in the encode methods of the static class `RobbersCipher` from the chapter on [fields](fields).
%```


```{exercise}
When and why would you choose to expose a parameter through the constructor as opposed to through an instance method?
Explain using your own words.
```


%```{exercise}
%TODO: First name and last name.
%```


%```{exercise-start}
%:label: ex:constructors:vowel
%```
%Below you'll find a draft of a class called `Weekday`.
%```{code-cell}
%class Weekday
%{
%  public int Day;
%  public Weekday (int day) { /* ... */ }
%  public Weekday (string day) { /* ... */ }
%  public Weekday (char day) { /* ... */ }
%}
%```
%Complete the definition of the class so that we can use it to run the following code and get the same result.
%```{code-cell} csharp
%class Weekday
%{
%  int day;
%
%  public Weekday (int day)
%  {
%    if (weekday < 1 || weekday > 7)
%      throw new ArgumentException("Number must be between 1 and 7.");
%  }
%
%  public Weekday (string day)
%  {
%    string[] legal = new string[] {
%      "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday" }
%    Array.IndexOf(legal, day)
%    if (legal.IndexOf(Char.ToUpper(vowel)) == -1)
%      throw new ArgumentException("Must be a vowel (AEIOU).");
%    letter = vowel;
%  }
%
%  public Weekday (int weekday)
%  {
%    if (weekday < 1 || weekday > 7)
%      throw new ArgumentException("Number must be between 1 and 7.");
%  }
%
%  public char ToChar () => letter;
%
%  public static implicit operator char(Vowel v) => v.ToChar();
%}
%```
%Write a class that models the domain concept of English vowels.
%It should be possible to pass upper and lowercase vowels as characters when instantiating it, and it should be possible to extract these vowels again.
%See the usage examples below.
%
%```{code-cell} csharp
%char c1 = new Vowel('A').ToChar();
%Console.WriteLine(c1);
%```
%```{code-cell} csharp
%char c2 = new Vowel('u').ToChar();
%Console.WriteLine(c2);
%```
%Rember the maxim [types over tests](types-over-tests)?
%It suggests that illegal states shouldn't be representable.
%If you try to instantiate the class by passing a non-vowel character then the class should throw an `ArgumentException`.
%
%```{code-cell} csharp
%:tags: [raises-exception]
%new Vowel('z').ToChar();
%```
%```{exercise-end}
%```


%```{exercise-start}
%```
%We have learned that illegal states should not be representable.
%We have also learned that compile-time errors are preferable over run-time errors.
%Rewrite the class `Weekday` that you wrote in {numref}`ex:constructors:vowel` so that we get a compile-time error instead of a run-time error whenever we try to instantiate it with a non-vowel.
%
%Hint: Use static factory methods.
%```{exercise-end}
%```

```{exercise}
Write a class that can be used to represent phone numbers for your country.
Rember the maxim [types over tests](types-over-tests).
Illegal states should not be representable.
```


%
%```{code-cell}
%class Vowel
%{
%  char vowel;
%
%  public Vowel (char vowel)
%  {
%    this.vowel = vowel;
%  }
%}
%```